In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.patches as mpatches

from itertools import izip

In [3]:
# this just makes a random dataframe, code stolen from some other notebook
wd = '/home/bay001/projects/tbos_clipseq_20160809/analysis/v3_8-15-2016'
bedhead = ['chrom','start','end','l10pval','l2fc','strand','ann','gene']
regions = ['3utr','5utr','CDS','5utr_and_3utr','intron','noncoding_exon','noncoding_intron','intergenic']
suffix = '.peaks.l2inputnormnew.bed.compressed.bed.annotated'

FOLD_CHANGE_CUTOFF = 3
PVALUE_CUTOFF = 5
import re
def get_counts(wd):
    samples = {}
    for f in glob.glob(os.path.join(wd,'*.annotated')):
        df = pd.read_table(f,names=bedhead)
        df = df[(df['l10pval'] > PVALUE_CUTOFF) & (df['l2fc'] > FOLD_CHANGE_CUTOFF)]
        basename = f.replace(wd+'/','').replace(suffix,'')
        basename = re.sub(r'\.basedon_\w+','',basename)
        samples[basename] = {}
        df['region'] = df.apply(return_region,axis=1)
        for key,value in df['region'].value_counts().iteritems():
            samples[basename][key] = value
        for region in regions:
            if region not in samples[basename]:
                samples[basename][region] = 0
    return samples
def return_region(row):
    region = row['ann'].split('|')[0]
    
    return region
# take out intergenic and 3_and_5prime annotations per discussion 8-25-2016
samples = get_counts(wd)
df = pd.DataFrame(samples)
df.drop('5utr_and_3utr',inplace=True)
df.drop('intergenic',inplace=True)
df = df.T

# this is just messing around with a maps function, but i'm too lazy to make a new notebook :) 
df2 = df.head(4)
df2.columns = range(0,6)
df2

,0,1,2,3,4,5
A11_01_01,1314,156,1562,231,82,62
A11_02_01,1088,263,1247,234,90,72
A1_01_01,2821,42,34,1592,140,62
A1_02_01,3177,45,36,1757,138,65


In [4]:
def remove_outliers(rbpdataframe, conf = 0.95):
    x = 0
    means = list()
    sems = list()
    for key, value in rbpdataframe.iteritems():
        df = rbpdataframe.dropna()
        
        nums = len(df[key])
        droppercent = (1-conf)/2.0
        dropnum = int(nums*(droppercent))
        # print(nums)
        # print(dropnum)
        df = df.sort_values(by=key)
        df.drop(df.head(dropnum).index, inplace=True)
        df.drop(df.tail(dropnum).index, inplace=True)
        # print(df[key])
        print(df)
        print(df[key].sem())
        means.append(df[key].mean())
        sems.append(df[key].sem())
    return means, sems

In [6]:
m, s = remove_outliers(df,0.5)
print(type(m))
print(type(s))

              3utr  5utr   CDS  intron  noncoding_exon  noncoding_intron
A3_01_01        27    75   870      98              26                 6
LUC_01_01       39   316  1417      79              32                19
A2_02_01        39    41    15     113               6                 2
A2_01_01        41    31    11     119               9                 2
LUC_02_01       55   323  1913      83              49                 2
WTL_02_01      125   240  4418     339             101                12
APO_02_01      166     9     4     209              21                 7
APO_01_01      193    11     6     249              26                 6
WTL_01_01      196   454  5993     376             152                21
DAZ_P_02_01    664     3     4     135              22                12
A11_02_01     1088   263  1247     234              90                72
DAZ_P_01_01   1165     8     4     294              33                18
A11_01_01     1314   156  1562     231             

In [7]:
test = pd.Series([1088,1314,2821])
print(test.sem())

543.926772032
